In [1]:
import pandas as pd
import numpy as np
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [2]:
df = pd.read_csv('D:/ML/Kaggle/playground-series-s5e5/train.csv')
df_test = pd.read_csv('D:/ML/Kaggle/playground-series-s5e5/test.csv')

In [3]:
df['BMI'] = df['Weight'] / ((df['Height']/100)**2)
df['Heart_rate_per_minute'] = df['Heart_Rate'] / df['Duration']
df['log_duration'] = np.log1p(df['Duration'])
df['log_hr'] = np.log1p(df['Heart_Rate'])
df['weight_height_ratio'] = df['Weight'] / df['Height']

## Test
df_test['BMI'] = df_test['Weight'] / ((df_test['Height']/100)**2)
df_test['Heart_rate_per_minute'] = df_test['Heart_Rate'] / df_test['Duration']
df_test['log_duration'] = np.log1p(df_test['Duration'])
df_test['log_hr'] = np.log1p(df_test['Heart_Rate'])
df_test['weight_height_ratio'] = df_test['Weight'] / df_test['Height']

In [4]:
df = df.drop(['id'], axis=1)

In [5]:
df_test2 = df_test.drop(['id'], axis=1)

In [6]:
df = pd.get_dummies(df)
df_test = pd.get_dummies(df_test)

In [8]:
print(df.columns)

Index(['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp',
       'Calories', 'BMI', 'Heart_rate_per_minute', 'log_duration', 'log_hr',
       'weight_height_ratio', 'Sex_female', 'Sex_male'],
      dtype='object')


In [9]:
predictors = ['Age', 'Height', 'Weight', 'Heart_Rate', 'Body_Temp', 'Heart_Rate', 'BMI', 'Heart_rate_per_minute', 'log_duration', 'log_hr',
       'weight_height_ratio', 'Sex_female', 'Sex_male']
response = 'Calories'

In [29]:
h2o.init(max_mem_size="4G") 

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 22.0.1+8-16, mixed mode, sharing)
  Starting server from D:\anaconda\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\mhkir\AppData\Local\Temp\tmpo8nqa4qb
  JVM stdout: C:\Users\mhkir\AppData\Local\Temp\tmpo8nqa4qb\h2o_mhkir_started_from_python.out
  JVM stderr: C:\Users\mhkir\AppData\Local\Temp\tmpo8nqa4qb\h2o_mhkir_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,6 months and 11 days
H2O_cluster_name:,H2O_from_python_mhkir_fychpn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.982 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [32]:
df_h2o= h2o.H2OFrame(df)
df_test_h2o = h2o.H2OFrame(df_test2)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
## GBM 
gbm_model= H2OGradientBoostingEstimator(nfolds=5, 
                             seed=1234, 
                             ntrees=250, 
                             max_depth=15,
                             keep_cross_validation_predictions=True)

In [15]:
gbm_model.train(x = predictors, y = response, training_frame=df_h2o)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1747111850659_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    88                 88                          8.15633e+06            0            15           14.6705       1             22029         7381.99

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 6.131385033867032
RMSE: 2.476163369785409
MAE: 1.6062928339435458
RMSLE: NaN
Mean Residual Deviance: 6.131385033867032

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 13.96296743776722
RMSE: 3.7367054256078602
MAE: 2.222132812504696
RMSLE: NaN
Mean Residual Deviance: 13.96296743776722

Cross-Validation Metrics Summary: 
                        mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan        0            nan           nan           nan           nan           nan
loglikelihood           nan        0            nan           nan           nan           nan           nan
mae                     2.22213    0.00792996   2.21665       2.23216       2.2181        2.21464       2.2291
mean_residual_deviance  13.9628    0.219052     13.9186       14.1168       14.0851       13.5974       14.0962
mse                     13.9628    0.219052     13.9186       14.1168       14.0851       13.5974       14.0962
r2                      0.996413   4.80369e-05  0.99643       0.996363      0.996398      0.996488      0.99639
residual_deviance       13.9628    0.219052     13.9186       14.1168       14.0851       13.5974       14.0962
rmse                    3.7366     0.0294251    3.73076       3.75724       3.75302       3.68747       3.7545
rmsle                   0.0624431  0.000184094  nan           0.0625553     0.0625435     0.0622307     nan

Scoring History: 
    timestamp            duration          number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------------  -----------------  ---------------  --------------  -------------------
    2025-05-13 07:58:53  3 min 13.113 sec  0                  62.3953          52.9295         3893.17
    2025-05-13 07:58:54  3 min 14.248 sec  1                  56.1769          47.6481         3155.84
    2025-05-13 07:58:55  3 min 15.051 sec  2                  50.582           42.8956         2558.54
    2025-05-13 07:58:56  3 min 15.721 sec  3                  45.5485          38.6192         2074.66
    2025-05-13 07:58:57  3 min 16.423 sec  4                  41.0202          34.7713         1682.66
    2025-05-13 07:58:57  3 min 17.091 sec  5                  36.9469          31.3091         1365.07
    2025-05-13 07:59:01  3 min 21.120 sec  11                 19.8168          16.73           392.707
    2025-05-13 07:59:06  3 min 25.573 sec  18                 9.82578          8.17174         96.546
    2025-05-13 07:59:10  3 min 30.195 sec  25                 5.30131          4.21488         28.1039
    2025-05-13 07:59:15  3 min 34.305 sec  33                 3.32204          2.39191         11.036
    2025-05-13 07:59:19  3 min 38.522 sec  42                 2.71516          1.78512         7.37208
    2025-05-13 07:59:23  3 min 42.675 sec  52                 2.56095          1.64245         6.55844
    2025-05-13 07:59:27  3 min 46.920 sec  63                 2.51196          1.61538         6.30993
    2025-05-13 07:59:31  3 min 51.249 sec  75                 2.4882           1.6083          6.19114
    2025-05-13 07:59:35  3 min 54.331 sec  88                 2.4

In [16]:
gbm_model.pd_plot

<bound method pd_plot of H2OGradientBoostingEstimator({'parms': {'model_id': {'__meta': {'schema_version': 3, 'schema_name': 'ModelParameterSchemaV3', 'schema_type': 'Iced'}, 'name': 'model_id', 'label': 'model_id', 'help': 'Destination id for this model; auto-generated if not specified.', 'required': False, 'type': 'Key<Model>', 'default_value': None, 'actual_value': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'GBM_model_python_1747111850659_1', 'type': 'Key<Model>', 'URL': '/3/Models/GBM_model_python_1747111850659_1'}, 'input_value': None, 'level': 'critical', 'values': [], 'is_member_of_frames': [], 'is_mutually_exclusive_with': [], 'gridable': False}, 'training_frame': {'__meta': {'schema_version': 3, 'schema_name': 'ModelParameterSchemaV3', 'schema_type': 'Iced'}, 'name': 'training_frame', 'label': 'training_frame', 'help': 'Id of the training data frame.', 'required': False, 'type': 'Key<Frame>', 'default_value': None, 'actu

In [ ]:
gbm_model2= H2OGradientBoostingEstimator(nfolds=5, 
                             seed=1234, 
                             ntrees=105, 
                             max_depth=15,
                             keep_cross_validation_predictions=True)

In [36]:
gbm_model2.train(x = predictors, y = response, training_frame=df_h2o)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1747114237605_3


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    88                 88                          8.15633e+06            0            15           14.6705       1             22029         7381.99

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 6.131385033867032
RMSE: 2.476163369785409
MAE: 1.6062928339435458
RMSLE: NaN
Mean Residual Deviance: 6.131385033867032

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 13.96296743776722
RMSE: 3.7367054256078602
MAE: 2.222132812504696
RMSLE: NaN
Mean Residual Deviance: 13.96296743776722

Cross-Validation Metrics Summary: 
                        mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan        0            nan           nan           nan           nan           nan
loglikelihood           nan        0            nan           nan           nan           nan           nan
mae                     2.22213    0.00792996   2.21665       2.23216       2.2181        2.21464       2.2291
mean_residual_deviance  13.9628    0.219052     13.9186       14.1168       14.0851       13.5974       14.0962
mse                     13.9628    0.219052     13.9186       14.1168       14.0851       13.5974       14.0962
r2                      0.996413   4.80369e-05  0.99643       0.996363      0.996398      0.996488      0.99639
residual_deviance       13.9628    0.219052     13.9186       14.1168       14.0851       13.5974       14.0962
rmse                    3.7366     0.0294251    3.73076       3.75724       3.75302       3.68747       3.7545
rmsle                   0.0624431  0.000184094  nan           0.0625553     0.0625435     0.0622307     nan

Scoring History: 
    timestamp            duration          number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------------  -----------------  ---------------  --------------  -------------------
    2025-05-13 08:43:22  3 min 36.831 sec  0                  62.3953          52.9295         3893.17
    2025-05-13 08:43:23  3 min 37.613 sec  1                  56.1769          47.6481         3155.84
    2025-05-13 08:43:24  3 min 38.307 sec  2                  50.582           42.8956         2558.54
    2025-05-13 08:43:24  3 min 38.984 sec  3                  45.5485          38.6192         2074.66
    2025-05-13 08:43:25  3 min 39.640 sec  4                  41.0202          34.7713         1682.66
    2025-05-13 08:43:26  3 min 40.283 sec  5                  36.9469          31.3091         1365.07
    2025-05-13 08:43:30  3 min 44.829 sec  12                 17.8848          15.0815         319.866
    2025-05-13 08:43:34  3 min 49.043 sec  19                 8.93123          7.39768         79.7669
    2025-05-13 08:43:39  3 min 53.493 sec  27                 4.58241          3.56734         20.9985
    2025-05-13 08:43:43  3 min 57.520 sec  35                 3.10172          2.17603         9.62064
    2025-05-13 08:43:47  4 min  1.561 sec  44                 2.66389          1.73448         7.09633
    2025-05-13 08:43:51  4 min  5.698 sec  54                 2.54718          1.63321         6.48814
    2025-05-13 08:43:55  4 min  9.735 sec  64                 2.51009          1.61455         6.30056
    2025-05-13 08:43:59  4 min 13.948 sec  75                 2.4882           1.6083          6.19114
    2025-05-13 08:44:03  4 min 17.103 sec  88                 2

In [39]:
rf_model = H2ORandomForestEstimator(ntrees=105, 
                                 max_depth=15, 
                                 nfolds=5,
                                 keep_cross_validation_predictions= True,
                                 seed=1234)

In [40]:
rf_model.train(x = predictors, y = response, training_frame=df_h2o)

drf Model Build progress: |

██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1747114237605_4


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    105                105                         3.37176e+07            15           15           15            20085         26951         25611.6

ModelMetricsRegression: drf
** Reported on train data. **

MSE: 14.156323954303689
RMSE: 3.7624890636789483
MAE: 2.262752818900532
RMSLE: 0.06247931937718695
Mean Residual Deviance: 14.156323954303689

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 14.086274248001496
RMSE: 3.7531685610962766
MAE: 2.252080791951724
RMSLE: 0.06231661695667261
Mean Residual Deviance: 14.086274248001496

Cross-Validation Metrics Summary: 
                        mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan        0            nan           nan           nan           nan           nan
loglikelihood           nan        0            nan           nan           nan           nan           nan
mae                     2.25208    0.0107254    2.24387       2.26136       2.24424       2.24491       2.266
mean_residual_deviance  14.0862    0.189639     13.8844       14.1603       14.1918       13.8895       14.3048
mse                     14.0862    0.189639     13.8844       14.1603       14.1918       13.8895       14.3048
r2                      0.996382   4.27578e-05  0.996439      0.996351      0.99637       0.996413      0.996336
residual_deviance       14.0862    0.189639     13.8844       14.1603       14.1918       13.8895       14.3048
rmse                    3.75309    0.0252765    3.72617       3.76302       3.7672        3.72686       3.78217
rmsle                   0.0623156  0.00037136   0.0618286     0.0621282     0.0627784     0.0622751     0.0625675

Scoring History: 
     timestamp            duration          number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------------  -----------------  ------------------  ------------------  -------------------
     2025-05-13 08:56:35  4 min 57.273 sec  0.0                nan                 nan                 nan
     2025-05-13 08:56:36  4 min 57.766 sec  1.0                5.0403228585972375  3.0056347388370623  25.404854518897828
     2025-05-13 08:56:36  4 min 58.266 sec  2.0                5.036820732682426   3.018745988324455   25.369563093179533
     2025-05-13 08:56:37  4 min 58.709 sec  3.0                4.919947899673717   2.9532725372076953  24.205887335503824
     2025-05-13 08:56:37  4 min 59.214 sec  4.0                4.743633455238919   2.8509431493072808  22.502058357661927
     2025-05-13 08:56:38  4 min 59.725 sec  5.0                4.643219011588716   2.7838485740508787  21.55948278957889
     2025-05-13 08:56:38  5 min  0.364 sec  6.0                4.560423650188002   2.736225176583983   20.797463869194065
     2025-05-13 08:56:39  5 min  0.875 sec  7.0                4.464413847664198   2.6915612969244265  19.930991003215848
     2025-05-13 08:56:43  5 min  5.041 sec  15.0               4.058812604255881   2.4551821743389675  16.47395975646641
     2025-05-13 08:56:47  5 min  9.305 sec  23.0               3.919099932566242   2.3707480374641694  15.359344281440723
---  ---                  ---               ---                ---                 ---                 ---
     2025-05-13 08:57:09  5 min 30.906 sec  58.0               3.803738657832853   2.291354015986667   14.468427777092076
     2025-05-13 08

In [41]:
ensemble =  H2OStackedEnsembleEstimator(model_id="model_combined", 
                                        base_models=[rf_model, gbm_model2])

In [42]:
ensemble.train(x = predictors, y = response, training_frame=df_h2o)

stackedensemble Model Build progress: |

██████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),2/2
# GBM base models (used / total),1/1
# DRF base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [43]:
pred= ensemble.predict(df_test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


d:\anaconda\Lib\site-packages\h2o\job.py:81: UserWarning: Test/Validation dataset is missing column 'Sex_female': substituting in a column of NaN
  warnings.warn(w)
d:\anaconda\Lib\site-packages\h2o\job.py:81: UserWarning: Test/Validation dataset is missing column 'Sex_male': substituting in a column of NaN
  warnings.warn(w)


In [44]:
pred.head()

predict
28.4039
107.008
87.1343
128.635
71.5805
20.8633
46.8576
6.87658
11.7392
206.043


In [45]:
pred_df = pred.as_data_frame()

d:\anaconda\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [48]:
submission = pd.DataFrame({'Calories':pred_df.iloc[:, 0]})

In [49]:
submission['id'] = df_test['id']

In [50]:
submission = submission[['id', 'Calories']]
submission.to_csv('submission3.csv', index=False)